
# Week 5: Object Detection
## Objective:
In this notebook, we will explore the basics of object detection, including key concepts such as Intersection over Union (IoU), precision-recall curves, and modern object detection models like YOLO. We will implement an object detection model and evaluate its performance on a sample dataset.



## 5.1 Theory: Introduction to Object Detection
Object detection is the task of detecting objects within an image and classifying them. Unlike image classification, object detection involves both identifying the object and localizing it within the image by drawing bounding boxes around detected objects.

### Key Metrics:
- **Intersection over Union (IoU)**: A metric used to evaluate the overlap between the predicted bounding box and the ground truth bounding box.
- **Precision-Recall Curve**: A tool used to evaluate the trade-off between precision and recall in object detection, especially when dealing with imbalanced data.

### Popular Object Detection Models:
- **YOLO (You Only Look Once)**: A real-time object detection model that processes the entire image in one pass, making it faster than traditional methods.
- **SSD (Single Shot Multibox Detector)**: A fast object detection model that eliminates region proposal stages and predicts bounding boxes directly.


In [6]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:

import cv2
import numpy as np

# Load YOLOv3 pre-trained weights and config
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO dataset class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load an image
img = cv2.imread("sample_image.jpg")
height, width, channels = img.shape

# Prepare the image for YOLO
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

# Run the detection
outs = net.forward(output_layers)

# Extract information from the outputs
class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Maximum Suppression (NMS) to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw bounding boxes
for i in indices:
    i = i[0]
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    label = str(classes[class_ids[i]])
    confidence = confidences[i]
    color = (0, 255, 0)
    cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Show the final image with detections
cv2.imshow("Object Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'



## 5.2 Intersection over Union (IoU)
IoU is used to measure the overlap between the predicted bounding box and the ground truth box. It is calculated as the area of overlap divided by the area of union.

$$ IoU = \frac{Area \ of \ Overlap}{Area \ of \ Union} $$

We will now implement a function to compute IoU.


In [2]:

# Function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate intersection
    inter_x1 = max(x1, x2)
    inter_y1 = max(y1, y2)
    inter_x2 = min(x1 + w1, x2 + w2)
    inter_y2 = min(y1 + h1, y2 + h2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

    # Calculate union
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    # Calculate IoU
    iou = inter_area / union_area
    return iou

# Example: Calculate IoU between two bounding boxes
boxA = [50, 50, 100, 100]
boxB = [60, 60, 100, 100]
iou = calculate_iou(boxA, boxB)
print(f"IoU: {iou:.2f}")


IoU: 0.68



## 5.3 Exercises:
- Experiment with different confidence thresholds for object detection and observe how it affects the number of detected objects.
- Implement a precision-recall curve for the YOLO model and evaluate its performance on a dataset.
- Try training a custom YOLO model on a small dataset and compare the results with a pre-trained model.
